# Predicting the sales of items from different stores for a russian company
## Tushar Pasricha - April 2020

In [ ]:
# Importing required libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Plotting Libraries
import seaborn as sns
import matplotlib.pyplot as plt


# settings
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#  Fetching all the files in pandas dataframes
shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
sales = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
item = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
item_catg = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
submission = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")

# **Data Exploration**

## **sales_train data** 

In [ ]:
sales.dtypes

In [ ]:
print('training data has {} columns and {} rows'.format(sales.shape[1],sales.shape[0]))
print('test data has {} columns and {} rows'.format(test.shape[1],test.shape[0]))

In [ ]:
#formatting the date column correctly
import datetime
sales.date=sales.date.apply(lambda x:datetime.datetime.strptime(x, '%d.%m.%Y'))

In [ ]:
sales.head()

item_cnt_day column has negative values as well, I will be considering them as the items returned to the stores on that day

In [ ]:
sales.describe()

## Checking the items which are present in test set but not in training set

In [ ]:
test_only = test[~test['item_id'].isin(sales['item_id'].unique())]['item_id'].unique()
print('test only items:', len(test_only))

In [ ]:
# dropping shops&items not in test data
shops_in_test = test.shop_id.unique()
items_in_test = test.item_id.unique()
sales = sales[sales.shop_id.isin(shops_in_test)]
sales = sales[sales.item_id.isin(items_in_test)]

print('sales:', sales.shape)

In [ ]:
sales.head()

# Checking for Outliers

In [ ]:
plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)
sns.boxplot(x=sales.item_cnt_day)

plt.figure(figsize=(10,4))
plt.xlim(sales.item_price.min(), sales.item_price.max()*1.1)
sns.boxplot(x=sales.item_price)

In the first Boxplot 
    1. There are 2 outliers

In the second Boxplot
    1. There is no outlier

## Removing the outliers

In [ ]:
sales = sales[sales.item_cnt_day<1000]

In [ ]:
plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)
sns.boxplot(x=sales.item_cnt_day)


plt.figure(figsize=(10,4))
plt.xlim(sales.item_price.min(), sales.item_price.max()*1.1)
sns.boxplot(x=sales.item_price)

In [ ]:
## Checking the number of items sold per month
month_wise_items_sold = sales.groupby('date_block_num').agg({'item_cnt_day' : sum},inplace = True , reset_index = True)
plt.figure(figsize=(18,5))
plt.title('Items sold by the company')
plt.xlabel('Time')
plt.ylabel('Items sold')
plt.plot(month_wise_items_sold);

## As we can see there is a sudden peak after the 10th month and 22nd month, that probably signifies the increase in sales during holidays period around christmas in Russia

In [ ]:
calculating_revenue = sales
calculating_revenue['revenue'] = calculating_revenue['item_price'] * calculating_revenue['item_cnt_day']
calculating_revenue.head()

In [ ]:
## Checking the revenue generated by the company per month
month_wise_revenue = calculating_revenue.groupby('date_block_num').agg({'revenue' : sum})
plt.figure(figsize=(20,5))
plt.title('revenue generated per month by the company')
plt.xlabel('Time')
plt.ylabel('Revenue generated')
plt.plot(month_wise_revenue);

## As we can see in the above graph the revenue generated is higher around 24th month in the series i.e December 2014 has the highest sales of all time

In [ ]:
shop_wise_revenue = calculating_revenue.groupby('shop_id').agg({'revenue' : sum})
plt.figure(figsize=(20,5))
plt.title('revenue generated by the individual shops')
plt.xlabel('Shop id')
plt.ylabel('Revenue generated')
plt.plot(shop_wise_revenue);

### Shop id 31 has the highest revenue amongst all

In [ ]:
## Checking number of returned items
returned_items = sales[sales['item_cnt_day'] < 0]
month_wise_items_returned = returned_items.groupby('date_block_num').agg({'item_cnt_day' : sum})
month_wise_items_returned['item_cnt_day'] = month_wise_items_returned['item_cnt_day'].abs()
plt.figure(figsize=(20,5))
plt.title('Items returned per month')
plt.xlabel('Time')
plt.ylabel('Items returned')
plt.plot(month_wise_items_returned);

### The maximum number of items were returned to the company in 24th month of the data provided i.e. December 2014

# Exploring Time Series

In [ ]:
timeseries = month_wise_items_sold 

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(timeseries.rolling(window=12,center=False).mean(),label='Rolling Mean');
plt.plot(timeseries.rolling(window=12,center=False).std(),label='Rolling sd');
plt.legend();

As we can see in the above graph, there is an obvious "seasonality" (Eg: peak sales around a time of year) and a increasing "Trend".

Let's check that with decomposition into Trend, seasonality and residuals.

In [ ]:
import statsmodels.api as sm

print('multiplicative model')
# multiplicative
mul_model = sm.tsa.seasonal_decompose(timeseries.values,freq=12,model="multiplicative")

fig = mul_model.plot()

# ------------------------------------------------------------------------------------
print('Additive model') 
# Additive
add_model = sm.tsa.seasonal_decompose(timeseries.values,freq=12,model="additive")

fig = add_model.plot()


## As we can see there is still some patterns left in the residual of additive model, we would assume the timeseries to be multiplicative that can be shown as yt=St x Tt x Et

In [ ]:
# Stationarity tests
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic

def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print('Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

test_stationarity(timeseries)

As p value > 0.05 we take the series as seosanal time series

## Decomposing time series data

In [ ]:
# To remove trend
from pandas import Series as Series

# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

# invert differenced forecast
def inverse_difference(last_ob, value):
    return value + last_ob

In [ ]:
timeseries.columns

In [ ]:
ts=sales.groupby(["date_block_num"])["item_cnt_day"].sum()
ts.astype('float')
plt.figure(figsize=(16,16))

plt.subplot(311)
plt.title('Original')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.plot(ts)

plt.subplot(312)
plt.title('After De-trend')
plt.xlabel('Time')
plt.ylabel('Sales')
new_ts=difference(ts)
plt.plot(new_ts)
plt.plot()

plt.subplot(313)
plt.title('After De-seasonalization')
plt.xlabel('Time')
plt.ylabel('Sales')
new_ts=difference(ts,12)       # assuming the seasonality is 12 months long
plt.plot(new_ts)
plt.plot()

In [ ]:
# now testing the stationarity again after de-seasonality
test_stationarity(new_ts)

As p value is now less than 0.05, we can consider that the data is stationary

# Time Series Forecasting
### We  will apply prophet model for baseline

In [ ]:
# adding the dates to the Time-series as index
ts=sales.groupby(["date_block_num"])["item_cnt_day"].sum()
ts.index=pd.date_range(start = '2013-01-01',end='2015-10-01', freq = 'MS')
ts=ts.reset_index()
ts.head()

In [ ]:
from fbprophet import Prophet

ts.columns=['ds','y']
#instantiate Prophet with only yearly seasonality as our data is monthly 
model = Prophet( yearly_seasonality=True) 
model.fit(ts) #fit the model with your dataframe

In [ ]:
# predict for five months in the furure and MS - month start is the frequency
future = model.make_future_dataframe(periods = 5, freq = 'MS')  
# now lets make the forecasts
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
model.plot(forecast)

In [ ]:
model.plot_components(forecast)